## Group Member’s Name:-
## Muhammad Arsalan (2303.KHI.DEG.025)
## Abdul Rehman (2303.KHI.DEG.035)
## Arshad Shiwani (2303.KHI.DEG.026)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import (max, min, avg, col, when, udf)
from pyspark.sql.types import StringType


## starting the spark session

In [2]:
spark = SparkSession.builder \
    .appName("Data Processing") \
    .getOrCreate()


23/05/29 11:00:55 WARN Utils: Your hostname, all-MS-7D35 resolves to a loopback address: 127.0.1.1; using 192.168.1.139 instead (on interface enp2s0)
23/05/29 11:00:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/29 11:00:56 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## given the columns name and import the data.

In [3]:
columns = ['PassengerId', 'Survived', 'Passenger_class','Name','Gender','Age','Siblings_Spouse','Par_child','Ticket','Fare','Cabin','Embarked','Timestamp']
              
df = spark.read.csv("./data/titanic.csv",inferSchema=True)
df = df.toDF(*columns)

df.show()


+-----------+--------+---------------+--------------------+------+----+---------------+---------+----------------+-------+-----+--------+-------------------+
|PassengerId|Survived|Passenger_class|                Name|Gender| Age|Siblings_Spouse|Par_child|          Ticket|   Fare|Cabin|Embarked|          Timestamp|
+-----------+--------+---------------+--------------------+------+----+---------------+---------+----------------+-------+-----+--------+-------------------+
|          1|       0|              3|Braund, Mr. Owen ...|  male|  22|              1|        0|       A/5 21171|   7.25| null|       S|2020-01-01 13:45:25|
|          2|       1|              1|Cumings, Mrs. Joh...|female|  38|              1|        0|        PC 17599|71.2833|  C85|       C|2020-01-01 13:44:48|
|          3|       1|              3|Heikkinen, Miss. ...|female|  26|              0|        0|STON/O2. 3101282|  7.925| null|       S|2020-01-01 13:38:11|
|          4|       1|              1|Futrelle, Mrs.

## Change the Coloums Values to Yes or No and 1 or 2 etc.

In [4]:
df = df.withColumn("Survived", when(col("Survived") == 1, "Yes").otherwise("No"))\
       .withColumn("Passenger_class", when(col("Passenger_class") == 1, "1st").when(col("Passenger_class") == 2, '2nd').otherwise('3rd'))\
       .withColumn("Embarked", when(col("Embarked") == 'S', 'Southampton').when(col("Embarked") == 'C', 'Cherbourg').otherwise('Queenstown'))

In [5]:
df.show()

+-----------+--------+---------------+--------------------+------+----+---------------+---------+----------------+-------+-----+-----------+-------------------+
|PassengerId|Survived|Passenger_class|                Name|Gender| Age|Siblings_Spouse|Par_child|          Ticket|   Fare|Cabin|   Embarked|          Timestamp|
+-----------+--------+---------------+--------------------+------+----+---------------+---------+----------------+-------+-----+-----------+-------------------+
|          1|      No|            3rd|Braund, Mr. Owen ...|  male|  22|              1|        0|       A/5 21171|   7.25| null|Southampton|2020-01-01 13:45:25|
|          2|     Yes|            1st|Cumings, Mrs. Joh...|female|  38|              1|        0|        PC 17599|71.2833|  C85|  Cherbourg|2020-01-01 13:44:48|
|          3|     Yes|            3rd|Heikkinen, Miss. ...|female|  26|              0|        0|STON/O2. 3101282|  7.925| null|Southampton|2020-01-01 13:38:11|
|          4|     Yes|            

## Now finding the Min,Max,and Avg here.

In [6]:
numerical_columns = [column[0] for column in df.dtypes if column[1] in ['int','float']]

numerical_df = df.select(*numerical_columns).describe()
summary_df = numerical_df.select(numerical_columns[1:]).summary('min','max','mean')




In [7]:
summary_df.show()

+-------+------------------+------------------+-----------------+
|summary|               Age|   Siblings_Spouse|        Par_child|
+-------+------------------+------------------+-----------------+
|    min|                 0|                 0|                0|
|    max|                80|               891|              891|
|   mean|167.64315089562422|180.12515025772694|179.6375301872114|
+-------+------------------+------------------+-----------------+



## Here we Make the user define Function in which we change the last alphabets to 1.

In [8]:
def change_last_letter(word):
    return word[:-1] + '1'

change_last_letter_udf = udf(change_last_letter, StringType())

categorical_columns = ['Survived', 'Passenger_class', 'Embarked']  # Replace with your categorical column names

for col in categorical_columns:
    df = df.withColumn(col, change_last_letter_udf(col))

df.show()


+-----------+--------+---------------+--------------------+------+----+---------------+---------+----------------+-------+-----+-----------+-------------------+
|PassengerId|Survived|Passenger_class|                Name|Gender| Age|Siblings_Spouse|Par_child|          Ticket|   Fare|Cabin|   Embarked|          Timestamp|
+-----------+--------+---------------+--------------------+------+----+---------------+---------+----------------+-------+-----+-----------+-------------------+
|          1|      N1|            3r1|Braund, Mr. Owen ...|  male|  22|              1|        0|       A/5 21171|   7.25| null|Southampto1|2020-01-01 13:45:25|
|          2|     Ye1|            1s1|Cumings, Mrs. Joh...|female|  38|              1|        0|        PC 17599|71.2833|  C85|  Cherbour1|2020-01-01 13:44:48|
|          3|     Ye1|            3r1|Heikkinen, Miss. ...|female|  26|              0|        0|STON/O2. 3101282|  7.925| null|Southampto1|2020-01-01 13:38:11|
|          4|     Ye1|            

## In this line of code we make parquet file in which we save first column data which is sorted.

In [9]:
# Assuming you have a DataFrame called 'df'
sorted_df = df.orderBy(df.columns[0])  # Sort by the first column

sorted_df.write.parquet("titanic_sorted_data_file.parquet")  # Save sorted DataFrame to Parquet file
